In [51]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import CLIPProcessor, CLIPModel, AutoProcessor, AutoModel


In [52]:
torch.set_default_device("cuda")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [53]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # CLIP expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])  # CLIP normalization
])


In [54]:
from torch.utils.data import Subset
cifar_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

In [55]:
class_names = cifar_data.classes  # CIFAR-10 class names
text_inputs = processor(text=class_names, return_tensors="pt", padding=True)


In [56]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

def denormalize(img: torch.Tensor, mean: torch.Tensor, std: torch.Tensor):
    """Denormalizes the image given the mean and standard deviation."""
    return img * torch.tensor(std, device="cpu").view(3, 1, 1) + torch.tensor(mean, device="cpu").view(3, 1, 1)

In [57]:
import numpy as np
from itertools import product

seeds = list(range(10))
results = []

for seed in seeds:

    fractions = [0.9, 0.1]
    total_len = len(cifar_data)
    lengths = [int(f * total_len) for f in fractions]
    lengths[-1] = total_len - sum(lengths[:-1])
    generator = torch.Generator(device="cuda").manual_seed(seed)

    # Split the dataset
    test_data, calib_data = torch.utils.data.random_split(cifar_data, lengths, generator=generator)
    calib_loader = DataLoader(calib_data, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

    # Forward pass through CLIP

    all_labels = []
    all_predictions = []
    scores = []

    # Compute nonconformity scores

    for images, labels in calib_loader:

        pil_images = [transforms.ToPILImage()(denormalize(img, processor.image_processor.image_mean, processor.image_processor.image_std)) for img in images]
        
        # Process images using CLIP's processor (automatically normalizes them)
        inputs = processor(images=pil_images, return_tensors="pt").to("cuda")
        input_image_processed = inputs['pixel_values'].squeeze(0)

        outputs = model(**inputs, **text_inputs)
        logits_per_image = outputs.logits_per_image  # Image-to-text similarity scores
        probs = logits_per_image.softmax(dim=1)  # Convert to probabilities
        predictions = probs.argmax(dim=1)
        all_labels.extend(labels.tolist())
        all_predictions.extend(predictions.tolist())
        scores += logits_per_image.take_along_dim(torch.tensor(labels).unsqueeze(-1),dim=1).squeeze().tolist()
        


    print(scores)



    alphas = [0.02, 0.05, 0.1, 0.2]
    for alpha in alphas:
        # Compute the quantile for the nonconformity scores
        n = len(scores)
        threshold = np.quantile(scores, np.ceil((n+1)*(alpha))/n, method="inverted_cdf")
        prediction_sets = []
        all_labels = []
        all_predictions = []

        for images, labels in test_loader:

            pil_images = [transforms.ToPILImage()(denormalize(img, processor.image_processor.image_mean, processor.image_processor.image_std)) for img in images]
            
            # Process images using CLIP's processor (automatically normalizes them)
            inputs = processor(images=pil_images, return_tensors="pt").to("cuda")
            input_image_processed = inputs['pixel_values'].squeeze(0)

            outputs = model(**inputs, **text_inputs)
            logits_per_image = outputs.logits_per_image  # Image-to-text similarity scores
            probs = logits_per_image.softmax(dim=1)  # Convert to probabilities
            predictions = probs.argmax(dim=1)
            all_labels.extend(labels.tolist())
            all_predictions.extend(predictions.tolist())
            indices = (logits_per_image > threshold).nonzero(as_tuple=True)
            row_indices = [indices[1][indices[0] == i] for i in range(logits_per_image.size(0))]
            prediction_sets.extend(row_indices)

        pred_sets = [x.tolist() for x in prediction_sets]
        coverage = np.mean([all_labels[i] in pred_sets[i] for i in range(len(all_labels))])
        avg_set_size = np.mean([len(s) for s in pred_sets])
        median_set_size = np.median([len(s) for s in pred_sets])
        acc_score = accuracy_score(all_labels, all_predictions)
        results.append([seed, alpha, coverage, avg_set_size, median_set_size, acc_score])
        # print(f"accuracy =\t\t {acc_score}")
        # print(f"coverage =\t\t {coverage}")
        # print(f"mean set size =\t\t {avg_set_size}")
        # print(f"median set size =\t {median_set_size}")

        print(seed,len(calib_data), len(test_data))

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


[25.252965927124023, 30.183380126953125, 26.752796173095703, 31.09225082397461, 29.038806915283203, 26.53801155090332, 29.167356491088867, 27.731943130493164, 26.997941970825195, 25.050024032592773, 27.23029899597168, 27.58633804321289, 27.784029006958008, 29.511795043945312, 29.560592651367188, 27.244945526123047, 26.311054229736328, 28.24018096923828, 28.928226470947266, 29.320680618286133, 28.326383590698242, 28.0279483795166, 27.81427001953125, 28.319622039794922, 28.82791519165039, 28.410879135131836, 26.64170265197754, 25.915267944335938, 28.487751007080078, 29.502857208251953, 23.946657180786133, 29.691696166992188, 27.95136833190918, 27.778200149536133, 28.544898986816406, 24.80379867553711, 29.2397403717041, 28.111839294433594, 27.02745819091797, 25.928884506225586, 27.475208282470703, 29.77061653137207, 27.922025680541992, 28.84884262084961, 30.116012573242188, 29.483917236328125, 28.033241271972656, 26.00082778930664, 29.474523544311523, 29.419681549072266, 25.58819007873535

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


[27.71703338623047, 29.552125930786133, 26.67011833190918, 29.140979766845703, 23.742088317871094, 29.541954040527344, 28.838624954223633, 28.09546661376953, 26.370080947875977, 28.586206436157227, 22.781124114990234, 26.009660720825195, 28.896156311035156, 23.787492752075195, 23.82308006286621, 30.06288719177246, 25.226852416992188, 24.237815856933594, 30.233034133911133, 27.749902725219727, 28.756174087524414, 28.84610939025879, 27.079326629638672, 26.591678619384766, 24.894052505493164, 28.559648513793945, 28.743946075439453, 23.5189266204834, 25.296815872192383, 29.87729263305664, 25.099014282226562, 29.132158279418945, 28.32421875, 29.933780670166016, 27.456172943115234, 26.751501083374023, 29.554134368896484, 27.800025939941406, 28.29216766357422, 26.38034439086914, 27.758121490478516, 29.41254997253418, 24.55747413635254, 26.057260513305664, 24.107805252075195, 28.882963180541992, 26.901206970214844, 28.3593807220459, 29.888227462768555, 29.54971694946289, 28.639713287353516, 26

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


...

[27.190723419189453, 29.183874130249023, 24.912893295288086, 29.69953727722168, 29.07370948791504, 26.647275924682617, 29.892576217651367, 24.72152328491211, 27.067211151123047, 29.262784957885742, 30.83416748046875, 26.80060577392578, 27.559412002563477, 24.28409767150879, 29.51938247680664, 26.201778411865234, 29.493314743041992, 23.912504196166992, 25.528156280517578, 27.846315383911133, 26.191091537475586, 27.850914001464844, 29.001602172851562, 22.787885665893555, 22.890884399414062, 27.04213523864746, 28.420642852783203, 25.655153274536133, 27.50726890563965, 23.474510192871094, 24.86570930480957, 30.536968231201172, 27.71233558654785, 25.27901840209961, 27.052425384521484, 27.230506896972656, 27.86720085144043, 29.369434356689453, 28.204185485839844, 28.06543731689453, 26.21415901184082, 29.42342185974121, 24.550662994384766, 29.838407516479492, 28.250810623168945, 24.08659553527832, 25.457611083984375, 28.97809410095215, 29.49164581298828, 24.2868709564209, 28.02889060974121, 2

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


[27.007814407348633, 29.29060173034668, 27.758121490478516, 29.255237579345703, 24.23723602294922, 25.605846405029297, 27.743022918701172, 26.009660720825195, 22.824153900146484, 29.43044662475586, 24.756011962890625, 26.811174392700195, 28.813631057739258, 29.836400985717773, 27.64727210998535, 28.67323112487793, 29.820329666137695, 26.916826248168945, 29.105836868286133, 30.989221572875977, 24.058731079101562, 26.900625228881836, 29.470699310302734, 28.17317771911621, 28.423368453979492, 30.780052185058594, 29.602140426635742, 28.012523651123047, 28.46643829345703, 30.649152755737305, 26.68900489807129, 27.698776245117188, 29.278139114379883, 29.03980827331543, 29.990779876708984, 27.973634719848633, 28.563356399536133, 30.811668395996094, 29.610624313354492, 28.783958435058594, 25.863039016723633, 29.120630264282227, 28.91157341003418, 28.9851016998291, 27.476146697998047, 24.063091278076172, 25.296588897705078, 29.64324378967285, 25.67626190185547, 23.586910247802734, 26.9385032653

/home/ra43rid/torch_plnet/venv/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


[26.69907569885254, 27.726703643798828, 24.79353904724121, 26.568456649780273, 29.708194732666016, 30.811012268066406, 27.700843811035156, 24.060087203979492, 27.48179817199707, 26.129606246948242, 29.385173797607422, 29.836400985717773, 26.792816162109375, 27.418798446655273, 28.909603118896484, 26.479413986206055, 28.510454177856445, 25.338972091674805, 26.45391845703125, 25.493419647216797, 27.70734977722168, 28.425857543945312, 23.77528190612793, 25.99851417541504, 27.070650100708008, 27.951631546020508, 23.72539710998535, 26.1348876953125, 30.496810913085938, 26.206663131713867, 27.437084197998047, 30.48931312561035, 25.336122512817383, 25.443401336669922, 29.995342254638672, 28.79241180419922, 29.482418060302734, 25.33019256591797, 24.492280960083008, 28.595130920410156, 28.245586395263672, 28.028955459594727, 28.017324447631836, 29.12392234802246, 28.270206451416016, 30.67861557006836, 29.501041412353516, 26.198200225830078, 27.627485275268555, 24.35097312927246, 25.492959976196

In [58]:
import pandas as pd
result_df = pd.DataFrame(data=results, columns=["seed", "alpha", "coverage", "avg_set_size", "median_set_size", "acc_score"])

In [59]:
result_df.to_csv("cifar10_clip_results.csv")

In [ ]:

agg = result_df.groupby('alpha').agg(['mean', 'std'])

# Step 2: Flatten MultiIndex columns
agg.columns = ['_'.join(col).strip() for col in agg.columns.values]

# Step 3: Format into "mean ± std" for each numeric column
formatted = pd.DataFrame(index=agg.index)

cols = [col for col in result_df.columns if col not in ["alpha", "seed", "acc_score", "median_set_size"]]

for col in cols:  # Skip 'Group'
    mean_col = f"{col}_mean"
    std_col = f"{col}_std"
    formatted[col] = agg[mean_col].round(4).astype(str) + ' ± ' + agg[std_col].round(4).astype(str)

print(formatted)


              coverage     avg_set_size median_set_size
alpha                                                  
0.02   0.9791 ± 0.0036   4.9845 ± 0.245    5.1 ± 0.3162
0.05   0.9492 ± 0.0054  3.5225 ± 0.1504       3.0 ± 0.0
0.10   0.8984 ± 0.0109  2.3808 ± 0.1465       2.0 ± 0.0
0.20   0.7957 ± 0.0117   1.4692 ± 0.061       1.0 ± 0.0


In [86]:
print(formatted.to_latex())

\begin{tabular}{llll}
\toprule
 & coverage & avg_set_size & median_set_size \\
alpha &  &  &  \\
\midrule
0.020000 & 0.9791 ± 0.0036 & 4.9845 ± 0.245 & 5.1 ± 0.3162 \\
0.050000 & 0.9492 ± 0.0054 & 3.5225 ± 0.1504 & 3.0 ± 0.0 \\
0.100000 & 0.8984 ± 0.0109 & 2.3808 ± 0.1465 & 2.0 ± 0.0 \\
0.200000 & 0.7957 ± 0.0117 & 1.4692 ± 0.061 & 1.0 ± 0.0 \\
\bottomrule
\end{tabular}

